In [1]:
pip install pygame

Note: you may need to restart the kernel to use updated packages.


In [9]:
import pygame
import random
import time

# Pygame Initialization
pygame.init()

# Constants
WIDTH, HEIGHT = 600, 400
GRID_SIZE = 10
TILE_SIZE = 40
MARGIN = 5
WHITE = (255, 255, 255)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
BLACK = (0, 0, 0)
GRAY = (169, 169, 169)

# Ship Information
SHIPS = [5, 4, 3, 3, 2]  # Carrier, Battleship, Cruiser, Submarine, Destroyer

# Load textures
tile_texture = pygame.image.load('tile_texture.jpg')  # Ensure you have this image
explosion_image = pygame.image.load('explosion.jpg')   # Ensure you have this image
hit_sound = pygame.mixer.Sound('hit_sound.wav')         # Ensure you have this sound
miss_sound = pygame.mixer.Sound('miss_sound.wav')       # Ensure you have this sound

# Set up the display
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Battleship with Monte Carlo AI")

# Create the board
player_grid = [[0] * GRID_SIZE for _ in range(GRID_SIZE)]
ai_grid = [[0] * GRID_SIZE for _ in range(GRID_SIZE)]

# Tracks AI guesses
ai_guesses = [[0] * GRID_SIZE for _ in range(GRID_SIZE)]

# Font for displaying turn information
font = pygame.font.SysFont("Arial", 24)

# Function to draw gradient background
def draw_gradient(surface, color1, color2):
    for i in range(HEIGHT):
        color = [
            int(color1[j] + (color2[j] - color1[j]) * (i / HEIGHT))
            for j in range(3)
        ]
        pygame.draw.line(surface, color, (0, i), (WIDTH, i))

# Function to draw the grids
def draw_grid(grid, offset_x, offset_y):
    for row in range(GRID_SIZE):
        for col in range(GRID_SIZE):
            if grid[row][col] == 1:
                color = RED  # Hit
            elif grid[row][col] == 2:
                color = BLUE  # Miss
            else:
                # Use texture for empty tiles
                screen.blit(tile_texture, 
                            [(MARGIN + TILE_SIZE) * col + offset_x,
                             (MARGIN + TILE_SIZE) * row + offset_y])
                continue
            
            pygame.draw.rect(screen, color, 
                             [(MARGIN + TILE_SIZE) * col + offset_x,
                              (MARGIN + TILE_SIZE) * row + offset_y, 
                              TILE_SIZE, TILE_SIZE])

# Place ships randomly on the board
def place_ships_randomly(grid):
    for ship in SHIPS:
        placed = False
        while not placed:
            direction = random.choice(['H', 'V'])  # Horizontal or Vertical
            if direction == 'H':
                row = random.randint(0, GRID_SIZE - 1)
                col = random.randint(0, GRID_SIZE - ship)
                if all(grid[row][c] == 0 for c in range(col, col + ship)):
                    for c in range(col, col + ship):
                        grid[row][c] = 3  # Ship marker
                    placed = True
            else:
                row = random.randint(0, GRID_SIZE - ship)
                col = random.randint(0, GRID_SIZE - 1)
                if all(grid[r][col] == 0 for r in range(row, row + ship)):
                    for r in range(row, row + ship):
                        grid[r][col] = 3  # Ship marker
                    placed = True

# Monte Carlo Simulation to guess the best move
def monte_carlo_simulation(ai_guesses, num_simulations=1000):
    score_grid = [[0] * GRID_SIZE for _ in range(GRID_SIZE)]

    for _ in range(num_simulations):
        temp_grid = [[0] * GRID_SIZE for _ in range(GRID_SIZE)]
        place_ships_randomly(temp_grid)

        for row in range(GRID_SIZE):
            for col in range(GRID_SIZE):
                if ai_guesses[row][col] == 0 and temp_grid[row][col] == 3:
                    score_grid[row][col] += 1

    # Find the max score cell
    max_score = 0
    best_move = (0, 0)
    for row in range(GRID_SIZE):
        for col in range(GRID_SIZE):
            if ai_guesses[row][col] == 0 and score_grid[row][col] > max_score:
                max_score = score_grid[row][col]
                best_move = (row, col)

    return best_move

# Check if all ships have been hit
def check_victory(grid):
    for row in grid:
        if 3 in row:  # Ship is still on the board
            return False
    return True

# Main Game Loop
def game_loop():
    place_ships_randomly(player_grid)
    place_ships_randomly(ai_grid)
    
    running = True
    player_turn = True
    game_over = False
    
    while running:
        draw_gradient(screen, (200, 200, 200), (100, 100, 100))  # Background gradient
        draw_grid(player_grid, 50, 50)
        draw_grid(ai_guesses, WIDTH // 2 + 50, 50)

        turn_text = font.render("Player's Turn" if player_turn else "AI's Turn", True, (0, 0, 0))
        screen.blit(turn_text, (10, 10))

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            if player_turn and not game_over:
                if event.type == pygame.MOUSEBUTTONDOWN:
                    pos = pygame.mouse.get_pos()
                    column = (pos[0] - WIDTH // 2 - 50) // (TILE_SIZE + MARGIN)
                    row = (pos[1] - 50) // (TILE_SIZE + MARGIN)

                    if 0 <= row < GRID_SIZE and 0 <= column < GRID_SIZE and ai_guesses[row][column] == 0:
                        if ai_grid[row][column] == 3:
                            ai_guesses[row][column] = 1  # Hit
                            hit_sound.play()
                        else:
                            ai_guesses[row][column] = 2  # Miss
                            miss_sound.play()

                        if check_victory(ai_grid):
                            game_over = True
                            print("Player wins!")

                        player_turn = False

            if player_turn and not game_over:
                # Highlight the tile being hovered over
                pos = pygame.mouse.get_pos()
                column = (pos[0] - WIDTH // 2 - 50) // (TILE_SIZE + MARGIN)
                row = (pos[1] - 50) // (TILE_SIZE + MARGIN)
                if 0 <= row < GRID_SIZE and 0 <= column < GRID_SIZE:
                    pygame.draw.rect(screen, (255, 255, 0), 
                                     [(MARGIN + TILE_SIZE) * column + WIDTH // 2 + 50, 
                                      (MARGIN + TILE_SIZE) * row + 50, 
                                      TILE_SIZE, TILE_SIZE], 3)  # Yellow border

        if not player_turn and not game_over:
            row, col = monte_carlo_simulation(ai_guesses)
            time.sleep(1)  # Simulate AI thinking

            if player_grid[row][col] == 3:
                player_grid[row][col] = 1  # Hit
                hit_sound.play()
            else:
                player_grid[row][col] = 2  # Miss
                miss_sound.play()

            if check_victory(player_grid):
                game_over = True
                print("AI wins!")

            player_turn = True

        pygame.display.flip()
    
    pygame.quit()

game_loop()
